# **Import Libraries**

In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
from tensorflow.keras.models import load_model # type: ignore
from PIL import Image, ImageTk
import pytesseract
import logging
import re
import os

# **Logging Setup**

In [8]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# **Load Trained CNN Model**

In [9]:
try:
    model = load_model('taka_detection_cnn_model.h5')
except Exception as e:
    logging.error(f"Error loading model: {e}")
    exit()

2025-04-15 19:56:55,574 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


# **Label Dictionary**

In [10]:
labels_dict = {
    0: '10 Taka',
    1: '20 Taka',
    2: '50 Taka',
    3: '100 Taka',
    4: '200 Taka',
    5: '500 Taka',
    6: '1000 Taka',
}

# **GUI Interface**

In [11]:
class BanknoteDetectorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Bangladeshi Taka Detector")
        self.root.geometry("1200x700")
        self.label = tk.Label(root, text="Bangladeshi Taka Detector", font=("Arial", 30), fg="yellow", bg="black")
        self.label.pack(pady=10)
        
        self.label = tk.Label(root, text="Select an image to Detect Bangladeshi Taka", font=("Times New Roman", 18), fg="red")
        self.label.pack(pady=10)

        self.select_button = tk.Button(
            root, 
            text="Select Image", 
            command=self.select_image, 
            font=("Arial", 14), 
            bg="green", 
            fg="white",  # White text for better contrast
            bd=0,  # No border
            relief="flat",  # Flat style
            highlightthickness=0,  # No focus highlight
            padx=10, pady=5  # Adds some padding for a better look
        )
        self.select_button.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.result_label = tk.Label(root, text="", font=("Arial", 30), fg="green")
        self.result_label.pack(pady=10)

        self.number_label = tk.Label(root, text="", font=("Arial", 12), fg="green")
        self.number_label.pack(pady=10)

    def select_image(self):
        file_path = filedialog.askopenfilename(
            title="Select Banknote Image",
            filetypes=[("Image files", "*.png *.jpg *.jpeg *.bmp")]
        )
        
        if not file_path:
            return

        image = cv2.imread(file_path, cv2.IMREAD_COLOR)
        if image is None:
            messagebox.showerror("Error", "Could not load the image!")
            return

        # Resize and preprocess for CNN
        cnn_input = cv2.resize(image, (224, 224))  #For CNN
        cnn_input = cnn_input / 255.0  #Normalization
        cnn_input = np.expand_dims(cnn_input, axis=0)  #Batch Dimension Add

        # Display image (original size)
        display_image = cv2.resize(image, (660, 380))
        
        # Predict with CNN
        probs = model.predict(cnn_input)[0]
        max_prob = np.max(probs)
        prediction = np.argmax(probs)
        print(f"Prediction index: {prediction}, Confidence: {max_prob}")
        print(f"Probabilities: {probs}")

        # Result text
        if max_prob >= 0.5:  # Confidence Threshold
            predicted_value = labels_dict.get(prediction, "Unknown")
            result_text = f"Detected: {predicted_value} (Confidence: {max_prob:.2f})"
            cv2.rectangle(display_image, (0, 0), (660, 380), (0, 255, 0), 3)  #Green Border
        else:
            result_text = "No Bangladeshi Taka Detected"


        # Convert image for Tkinter
        image_rgb = cv2.cvtColor(display_image, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(image_rgb)
        imgtk = ImageTk.PhotoImage(image=img)

        # Update GUI
        self.image_label.config(image=imgtk)
        self.image_label.image = imgtk
        self.result_label.config(text=result_text)

# **RUN GUI**

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = BanknoteDetectorApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Prediction index: 1, Confidence: 0.9534040093421936
Probabilities: [5.6879307e-10 9.5340401e-01 9.0437334e-07 6.7333413e-06 3.6562647e-05
 4.6551447e-02 4.3072529e-07]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction index: 0, Confidence: 0.9991914629936218
Probabilities: [9.9919146e-01 1.0078640e-08 2.4081901e-06 8.3078788e-08 5.2312808e-04
 2.6403004e-06 2.8022693e-04]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction index: 2, Confidence: 0.9991476535797119
Probabilities: [3.8924136e-06 7.0546841e-04 9.9914765e-01 1.1907237e-06 1.4067038e-04
 1.2230123e-07 1.0101859e-06]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction index: 2, Confidence: 0.9961701035499573
Probabilities: [1.3949508e-04 2.3952171e-03 9.9617010e-01 1.4248256e-05 1.2445853e-03
 4.0685941e-06 3.2256565e-05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction index: 3, Confidence: 0.9375013113021851
Probabilities: [1.5913769e-11 1.7304973e-07 7.2338054e-08 9.3750131e-01 8.3092935e-11
